# VK-API: GEOtagged photos


by Bella Mironova based on the [materials]("https://gainanov.pro/rus-blog/programming/vk-favourite-places-for-photo/") of Ruslan Gaynanov


## 0. Import libraries


In [6]:
import pandas as pd
import geopandas as gpd
import folium
import requests
from random import randint
from datetime import datetime
from time import sleep, strftime, localtime, mktime, strptime
from csv import DictWriter, DictReader
from folium.plugins import HeatMap

## 1. Select GEOgraphical area and timeperiod


In [7]:
coords = (60.7096, 28.7490)  # (latitude, longitude)

dist = 3000  # радиус поиска фотографий (в метрах)

timeperiod = (1640995200, 1643673600) # диапазон времени в котором сделано фото
                                      # ( 00:00 1.01.2022,  00:00 1.02.2022 )
vk_access_token = '34a9a2a734a9a2a734a9a2a70937bb1d32334a934a9a2a7575af68365af3b1d411ce236'

vk_version = 5.131

## 2. Search photos in a selected area and timeperiod


In [8]:

def getVK(geo, timeperiod, offset):
    params = {
        'lat': coords[0],
        'long': coords[1],
        'count': '1000',
        'offset': offset,
        'radius': dist,
        'start_time': timeperiod[0],
        'end_time': timeperiod[1],
        'access_token': vk_access_token,
        'v': vk_version,
        'sort': 0 # by date of creation
    }
    return requests.get("https://api.vk.com/method/photos.search",
                        params=params, verify=True).json()

## 3. Save response data


In [9]:
def save_to_map(resp, map_data):
    try:
        items = resp['response']['items']
    except KeyError:
        return
    for f in items:
        try:
            map_data[(f['id'], f['date'])] = (f['lat'], f['long'])
        except KeyError:
            continue

In [10]:
map_data = {}
step = 24*60*60 # шаг - день
i = timeperiod[0]
while i < timeperiod[1]:
    resp = getVK(coords, (i, i+step), 0)
    save_to_map(resp, map_data)
    count = resp['response']['count']
    returned = len(resp['response']['items'])
    if count > returned:
        offset = returned
        while offset < count and offset < 3000:
            resp = getVK(geo, (i, i+step), offset)
            save_to_map(resp, map_data)
            count = resp['response']['count']
            returned = len(resp['response']['items'])
            offset = offset + returned
            if returned == 0:
                break
    i = i + step
    sleep(0.5)      

In [6]:
def save_map_to_csv(map_data, path):
    with open(path, 'w', newline='', encoding='utf-8') as csvfile:
        columns = ['id', 'date', 'lat', 'long']
        writer = DictWriter(csvfile, fieldnames=columns, delimiter=';')
        writer.writeheader()
        for key in map_data:
            data = {
                'id': key[0],
                'date': strftime("%d.%m.%Y", localtime(key[1])),
                'lat': map_data[key][0],
                'long': map_data[key][1],
            }
            writer.writerow(data)

In [13]:
def get_map_to_df(map_data):
        columns = ['id', 'date', 'lat', 'long']
        map_df = pd.DataFrame()
        for key in map_data:
            data = {
                'id': key[0],
                'date': strftime("%d.%m.%Y", localtime(key[1])),
                'lat': map_data[key][0],
                'long': map_data[key][1],
            }
            data_df = pd.DataFrame.from_dict([data])
            map_df = pd.concat([map_df, data_df], axis=0)

        return  map_df
            

In [7]:
save_map_to_csv(map_data, 'vyborg2022_2.csv')

In [14]:
map_df = get_map_to_df(map_data)

In [15]:
map_df.head()

,id,date,lat,long
0,457273775,02.01.2022,60.695277,28.769146
0,457328838,02.01.2022,60.709996,28.748336
0,457328837,02.01.2022,60.709997,28.748874
0,457328836,02.01.2022,60.697512,28.781669
0,457328835,02.01.2022,60.697512,28.781669


In [16]:
spatial = gpd.GeoDataFrame(map_df, geometry=gpd.points_from_xy(map_df.long, map_df.lat), crs="EPSG:4326")

In [17]:
osm_map = folium.Map(location=[map_df.lat.mean(), map_df.long.mean()], zoom_start=10)


In [20]:
map_matrix = map_df[['lat', 'long']].values
HeatMap(map_matrix).add_to(osm_map)

NameError: name 'HeatMap' is not defined

In [19]:
osm_map